In [ ]:
!pip install gdal

In [ ]:
from glob import glob
import os
import json
import numpy as np
from osgeo import gdal
import scipy.misc as sm

import matplotlib.pyplot as plt


In [ ]:
def pad_year_day(year, day):
    day_str = str(day) + ""
    
    if(len(day_str) == 1):
        day_str = day_str.rjust(3, "0")   
    elif(len(day_str) == 2):
        day_str = day_str.rjust(3, "0")   

    return year + day_str

def get_bands_for_date(year, day):
    day_str = pad_year_day(year, day)
    dirs = os.listdir("./data/" + day_str)

    result = {}

    for f in dirs:
        if(f.endswith("qa.TIF")):
            continue
        elif(not f.endswith(".TIF")):
            continue
        band = f.split("_")[1].replace(".TIF", "")

        result[band] = os.path.join("./data", day_str,f)

    return result


In [ ]:
def extract_time_profile():
    day_range = range(1,300)
    year = "2019"

    band_data = {
    }

    # profiles[100, 100, 10] = 121
    # print(profiles)
    # print(profiles[100, 100, 10])

    #bands = ["B01", "B02", "B03", "B04", "B05", "B06", "B06", "B07"]
    bands = ["B02"]

    for b in bands:
        print("Band:", b)
        b_data = []
        for day in day_range:    
            file_names = get_bands_for_date(year, day)
            #print(file_names)
            if(b not in file_names):
                print("Band Name Not found")
                continue
            
            b_file_name = file_names[b]
            print(b_file_name)

            dataset = gdal.Open(b_file_name) 
            data = dataset.ReadAsArray().astype(np.float)
            n_data = data[5:15, 1300:1310]
            del dataset 
            #print(n_data)
            b_data.append(n_data)

        b_data_3d = np.dstack(b_data)

        band_data[b] = b_data_3d

    return band_data


In [ ]:
band_data = extract_time_profile()

In [ ]:
x_axis = []
for i in range(0, 99):
    x_axis.append(i)

band_map = {
    "B01": "VIS-RED",
    "B02": "NIR",
    "B03": "VIS-BLUE",
    "B04": "VIS-GREEN",
    "B05": "NIR",
    "B06": "SWIR",
    "B07": "SWIR"
}
    
#band_names = ["B01", "B02", "B03", "B04", "B05", "B06", "B07"]
band_names = ["B02"]


# 5,6 & 1300,1301
# 251,252 & 1567,1568
for bn in band_names:
    for x in range(4,6):
        for y in range(4,6):    
            
            plt.imshow(band_data[bn][0])
            
            time_series = band_data[bn][x, y, 1:100]
            
#             for k in range(0, len(time_series)):
#                 if(time_series[k] > 32700):
#                     time_series[k] = 1500
            
            f = plt.figure() 
            f.set_figwidth(20) 
            f.set_figheight(8) 
            plt.plot(x_axis, time_series)
            plt.xlabel('Day')
            plt.ylabel('NBAR')

            #plt.yscale('log')
            plt.title('Band {} {} -TimeSeries for Location: {} {}'.format(bn, band_map[bn], x,y))
            plt.show()

In [ ]:
for x in range(0, 2400):
    for y in range(0, 2400):    
        data = band_data["B01"][x, y, 1]
        if(data < 32767):
            print(x, y, data)
